In [1]:
import ray
from ray.util.queue import Queue

ray.init()

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-03-22_10-50-25_828518_24824/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-22_10-50-25_828518_24824/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-03-22_10-50-25_828518_24824',
 'metrics_export_port': 61529,
 'gcs_address': '127.0.0.1:63756',
 'address': '127.0.0.1:63756',
 'node_id': 'e7d7d21bf9ce52c28fe0908b018c4cd9f22883d10a901870d3d4f608'}

In [8]:
# You can pass this object around to different tasks/actors
queue = Queue(maxsize=100)
import time

@ray.remote
def consumer(queue,i):
    while queue.empty():
        1
    while not queue.empty():
        time.sleep(0.1)
        try:
            next_item = queue.get(block=False)
            print(f"Consumer {i} got work {next_item}")
        except Empty:
            print('Queue for process %d is empty'%i)

@ray.remote
def producer(queue,i):
    j=0
    while not queue.full():
        time.sleep(0.1)
        try:
            item=(i,j)
            j+=1
            if j>4:
                break
            queue.put(item,block=True)
            print(f"producer {i} put {item}")
        except full:
            print('Queue for producer %d is full (should not happen because of block)'%i)

producers = [producer.remote(queue,i) for i in range(3)]

consumers = [consumer.remote(queue,i) for i in range(2)]

(producer pid=24861) producer 2 put (2, 0)
(producer pid=32994) producer 1 put (1, 0)
(producer pid=32992) producer 0 put (0, 0)
(consumer pid=24862) Consumer 1 got work (0, 0)
(consumer pid=24860) Consumer 0 got work (1, 0)
(producer pid=24861) producer 2 put (2, 1)
(producer pid=32994) producer 1 put (1, 1)
(producer pid=32992) producer 0 put (0, 1)
(consumer pid=24862) Consumer 1 got work (2, 0)
(consumer pid=24860) Consumer 0 got work (0, 1)
(producer pid=24861) producer 2 put (2, 2)
(producer pid=32994) producer 1 put (1, 2)
(producer pid=32992) producer 0 put (0, 2)
(consumer pid=24862) Consumer 1 got work (2, 1)
(consumer pid=24860) Consumer 0 got work (1, 1)
(producer pid=24861) producer 2 put (2, 3)
(producer pid=32994) producer 1 put (1, 3)
(producer pid=32992) producer 0 put (0, 3)
(consumer pid=24862) Consumer 1 got work (1, 2)
(consumer pid=24860) Consumer 0 got work (0, 2)
(consumer pid=24862) Consumer 1 got work (0, 3)
(consumer pid=24860) Consumer 0 got work (2, 2)
(con